# Import and sandbox

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools
import yaml

In [5]:
df = pd.read_parquet("/home/maxime/synrepos/phenospace/jump_pl/images_embeddings/ResNet50/metadata/load_wells/load_wells.parquet")

In [6]:
plates = df['Metadata_Plate'].unique()
for plate in tqdm(plates):
    df_plate= df[(df['Metadata_Plate']==plate) & (df['Metadata_JCP2022']=='JCP2022_033924')]
    if len(df_plate)

,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_InChI,Metadata_PlateType,Metadata_Batch,Well_Embedding_mean,Well_Embedding_median,Metadata_Row,Metadata_Col,Metadata_JCP2022
0,source_1,UL000081,A02,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,COMPOUND,Batch4_20221012,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,A,02,JCP2022_033924
1,source_1,UL000081,A03,InChI=1S/C17H30N2O5/c1-6-23-17(22)14-13(24-14)...,COMPOUND,Batch4_20221012,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,A,03,JCP2022_085227
2,source_1,UL000081,A04,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,COMPOUND,Batch4_20221012,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,A,04,JCP2022_033924
3,source_1,UL000081,A05,InChI=1S/C23H18FNO4S/c1-2-10-29-16-5-3-4-14(11...,COMPOUND,Batch4_20221012,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,A,05,JCP2022_047857
4,source_1,UL000081,A06,InChI=1S/C19H24N2O5/c1-5-21(19(22)12-6-7-12)11...,COMPOUND,Batch4_20221012,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,A,06,JCP2022_072229
...,...,...,...,...,...,...,...,...,...,...,...
746870,source_9,GR00004421,Z24,InChI=1S/C28H21N7OS/c1-17-15-24(37-16-17)25-20...,COMPOUND,20211103-Run16,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,Z,24,JCP2022_037716
746871,source_9,GR00004421,Z25,InChI=1S/C28H21N7OS/c1-17-15-24(37-16-17)25-20...,COMPOUND,20211103-Run16,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,Z,25,JCP2022_037716
746872,source_9,GR00004421,Z26,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,COMPOUND,20211103-Run16,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,Z,26,JCP2022_033924
746873,source_9,GR00004421,Z47,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,COMPOUND,20211103-Run16,/home/maxime/synrepos/phenospace/jump_pl/image...,/home/maxime/synrepos/phenospace/jump_pl/image...,Z,47,JCP2022_033924


In [8]:

df['Metadata_JCP2022'].value_counts()

Metadata_JCP2022
JCP2022_033924    81340
JCP2022_037716     7666
JCP2022_064022     7663
JCP2022_025848     7660
JCP2022_046054     7654
                  ...  
JCP2022_010836        2
JCP2022_044729        2
JCP2022_100016        2
JCP2022_080384        1
JCP2022_090832        1
Name: count, Length: 112480, dtype: int64